<a href="https://colab.research.google.com/github/FlyAIBox/Agent_In_Action/blob/main/live-streaming/05-llm-inference/deepseek_vllm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 基于vLLM部署DeepSeek-R1-Distill-Qwen-1.5B
## 📖 关于 DeepSeek R1 蒸馏版 Qwen 1.5B 模型

### 🧠 模型特点
- **模型名称**: `deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B`
- **参数规模**: 15亿参数，适合在显存有限的GPU（如Nvidia T4/4090） 上运行
- **推理能力**: 继承了 DeepSeek R1 的强大推理能力
- **蒸馏技术**: 通过知识蒸馏获得更小但高效的模型

### 🔍 模型优势
1. **轻量化**: 15亿参数，内存占用小
2. **高效推理**: 优化的推理速度
3. **强大能力**: 保持了大模型的推理能力
4. **免费部署**: 适合在 Colab上免费的Nvidia T4 GPU 上运行

## 🎯 实验目标

本实验旨在帮助大模型技术初学者：

### 📚 学习内容
1. **环境准备**: 了解如何检查和配置 Colab 环境
2. **依赖安装**: 学习安装 VLLM等关键库
3. **模型部署**: 掌握使用 VLLM 部署大语言模型的方法


### 💰 成本优势
- **完全免费**: 使用 Google Colab 免费 T4 GPU (15GB 显存)
- **零配置**: 无需本地环境配置，浏览器即可运行
- **即开即用**: 一键启动，快速体验大模型部署

### 🚀 期望收获
通过本实验，你将掌握：
- 大语言模型的基本部署流程
- VLLM 推理引擎的使用方法

## 🔧 第一步：环境信息检查

在开始部署模型之前，我们需要了解当前的运行环境。这个步骤非常重要，因为：

### 🎯 检查目的
1. **硬件确认**: 确保有足够的 GPU 显存运行模型
2. **系统兼容**: 验证操作系统和 Python 版本
3. **资源评估**: 了解可用的 CPU、内存和存储空间
4. **环境配置**: 检查 CUDA 版本和相关依赖

### 📊 检查内容
- **操作系统**: Linux 发行版和版本
- **CPU 信息**: 处理器型号和核心数
- **内存状态**: 总内存和可用内存
- **GPU 配置**: 显卡型号和显存大小
- **CUDA 版本**: 深度学习框架支持
- **Python 环境**: 解释器版本
- **磁盘空间**: 可用存储空间

In [1]:
# 🔍 环境信息检查脚本
#
# 本脚本的作用：
# 1. 安装 pandas 库用于数据表格展示
# 2. 检查系统的各项配置信息
# 3. 生成详细的环境报告表格
#
# 对于初学者来说，这个步骤帮助你：
# - 了解当前运行环境的硬件配置
# - 确认是否满足模型运行的最低要求
# - 学习如何通过代码获取系统信息

# 安装 pandas 库 - 用于创建和展示数据表格
# pandas 是 Python 中最流行的数据处理和分析库
!pip install pandas==2.2.2

import platform # 导入 platform 模块以获取系统信息
import os # 导入 os 模块以与操作系统交互
import subprocess # 导入 subprocess 模块以运行外部命令
import pandas as pd # 导入 pandas 模块，通常用于数据处理，这里用于创建表格
import shutil # 导入 shutil 模块以获取磁盘空间信息

# 获取 CPU 信息的函数，包括核心数量
def get_cpu_info():
    cpu_info = "" # 初始化 CPU 信息字符串
    physical_cores = "N/A"
    logical_cores = "N/A"

    if platform.system() == "Windows": # 如果是 Windows 系统
        cpu_info = platform.processor() # 使用 platform.processor() 获取 CPU 信息
        try:
            # 获取 Windows 上的核心数量 (需要 WMI)
            import wmi
            c = wmi.WMI()
            for proc in c.Win32_Processor():
                physical_cores = proc.NumberOfCores
                logical_cores = proc.NumberOfLogicalProcessors
        except:
            pass # 如果 WMI 不可用，忽略错误

    elif platform.system() == "Darwin": # 如果是 macOS 系统
        # 在 macOS 上使用 sysctl 命令获取 CPU 信息和核心数量
        os.environ['PATH'] = os.environ['PATH'] + os.pathsep + '/usr/sbin' # 更新 PATH 环境变量
        try:
            process_brand = subprocess.Popen(['sysctl', "machdep.cpu.brand_string"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            stdout_brand, stderr_brand = process_brand.communicate()
            cpu_info = stdout_brand.decode().split(': ')[1].strip() if stdout_brand else "Could not retrieve CPU info"

            process_physical = subprocess.Popen(['sysctl', "hw.physicalcpu"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            stdout_physical, stderr_physical = process_physical.communicate()
            physical_cores = stdout_physical.decode().split(': ')[1].strip() if stdout_physical else "N/A"

            process_logical = subprocess.Popen(['sysctl', "hw.logicalcpu"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            stdout_logical, stderr_logical = process_logical.communicate()
            logical_cores = stdout_logical.decode().split(': ')[1].strip() if stdout_logical else "N/A"

        except:
            cpu_info = "Could not retrieve CPU info"
            physical_cores = "N/A"
            logical_cores = "N/A"

    else:  # Linux 系统
        try:
            # 在 Linux 上读取 /proc/cpuinfo 文件获取 CPU 信息和核心数量
            with open('/proc/cpuinfo') as f:
                physical_cores_count = 0
                logical_cores_count = 0
                cpu_info_lines = []
                for line in f:
                    if line.startswith('model name'): # 查找以 'model name'开头的行
                        if not cpu_info: # 只获取第一个 model name
                            cpu_info = line.split(': ')[1].strip()
                    elif line.startswith('cpu cores'): # 查找以 'cpu cores' 开头的行
                        physical_cores_count = int(line.split(': ')[1].strip())
                    elif line.startswith('processor'): # 查找以 'processor' 开头的行
                        logical_cores_count += 1
                physical_cores = str(physical_cores_count) if physical_cores_count > 0 else "N/A"
                logical_cores = str(logical_cores_count) if logical_cores_count > 0 else "N/A"
                if not cpu_info:
                     cpu_info = "Could not retrieve CPU info"

        except:
            cpu_info = "Could not retrieve CPU info"
            physical_cores = "N/A"
            logical_cores = "N/A"

    return f"{cpu_info} ({physical_cores} physical cores, {logical_cores} logical cores)" # 返回 CPU 信息和核心数量


# 获取内存信息的函数
def get_memory_info():
    mem_info = "" # 初始化内存信息字符串
    if platform.system() == "Windows":
        # 在 Windows 上不容易通过标准库获取，需要外部库或 PowerShell
        mem_info = "Requires external tools on Windows" # 设置提示信息
    elif platform.system() == "Darwin": # 如果是 macOS 系统
        # 在 macOS 上使用 sysctl 命令获取内存大小
        process = subprocess.Popen(['sysctl', "hw.memsize"], stdout=subprocess.PIPE, stderr=subprocess.PIPE) # 运行 sysctl 命令
        stdout, stderr = process.communicate() # 获取标准输出和标准错误
        mem_bytes = int(stdout.decode().split(': ')[1].strip()) # 解析输出，获取内存大小（字节）
        mem_gb = mem_bytes / (1024**3) # 转换为 GB
        mem_info = f"{mem_gb:.2f} GB" # 格式化输出
    else:  # Linux 系统
        try:
            # 在 Linux 上读取 /proc/meminfo 文件获取内存信息
            with open('/proc/meminfo') as f:
                total_mem_kb = 0
                available_mem_kb = 0
                for line in f:
                    if line.startswith('MemTotal'): # 查找以 'MemTotal' 开头的行
                        total_mem_kb = int(line.split(':')[1].strip().split()[0]) # 解析行，获取总内存（KB）
                    elif line.startswith('MemAvailable'): # 查找以 'MemAvailable' 开头的行
                         available_mem_kb = int(line.split(':')[1].strip().split()[0]) # 解析行，获取可用内存（KB）

                if total_mem_kb > 0:
                    total_mem_gb = total_mem_kb / (1024**2) # 转换为 GB
                    mem_info = f"{total_mem_gb:.2f} GB" # 格式化输出总内存
                    if available_mem_kb > 0:
                        available_mem_gb = available_mem_kb / (1024**2)
                        mem_info += f" (Available: {available_mem_gb:.2f} GB)" # 添加可用内存信息
                else:
                     mem_info = "Could not retrieve memory info" # 如果读取文件出错，设置错误信息

        except:
            mem_info = "Could not retrieve memory info" # 如果读取文件出错，设置错误信息
    return mem_info # 返回内存信息

# 获取 GPU 信息的函数，包括显存
def get_gpu_info():
    try:
        # 尝试使用 nvidia-smi 获取 NVIDIA GPU 信息和显存
        result = subprocess.run(['nvidia-smi', '--query-gpu=name,memory.total', '--format=csv,noheader'], capture_output=True, text=True)
        if result.returncode == 0: # 如果命令成功执行
            gpu_lines = result.stdout.strip().split('\n') # 解析输出，获取 GPU 名称和显存
            gpu_info_list = []
            for line in gpu_lines:
                name, memory = line.split(', ')
                gpu_info_list.append(f"{name} ({memory})") # 格式化 GPU 信息
            return ", ".join(gpu_info_list) if gpu_info_list else "NVIDIA GPU found, but info not listed" # 返回 GPU 信息或提示信息
        else:
             # 尝试使用 lshw 获取其他 GPU 信息 (需要安装 lshw)
            try:
                result_lshw = subprocess.run(['lshw', '-C', 'display'], capture_output=True, text=True)
                if result_lshw.returncode == 0: # 如果命令成功执行
                     # 简单解析输出中的 product 名称和显存
                    gpu_info_lines = []
                    current_gpu = {}
                    for line in result_lshw.stdout.splitlines():
                        if 'product:' in line:
                             if current_gpu:
                                 gpu_info_lines.append(f"{current_gpu.get('product', 'GPU')} ({current_gpu.get('memory', 'N/A')})")
                             current_gpu = {'product': line.split('product:')[1].strip()}
                        elif 'size:' in line and 'memory' in line:
                             current_gpu['memory'] = line.split('size:')[1].strip()

                    if current_gpu: # 添加最后一个 GPU 的信息
                        gpu_info_lines.append(f"{current_gpu.get('product', 'GPU')} ({current_gpu.get('memory', 'N/A')})")

                    return ", ".join(gpu_info_lines) if gpu_info_lines else "GPU found (via lshw), but info not parsed" # 如果找到 GPU 但信息无法解析，设置提示信息
                else:
                    return "No GPU found (checked nvidia-smi and lshw)" # 如果两个命令都找不到 GPU，设置提示信息
            except FileNotFoundError:
                 return "No GPU found (checked nvidia-smi, lshw not found)" # 如果找不到 lshw 命令，设置提示信息
    except FileNotFoundError:
        return "No GPU found (nvidia-smi not found)" # 如果找不到 nvidia-smi 命令，设置提示信息


# 获取 CUDA 版本的函数
def get_cuda_version():
    try:
        # 尝试使用 nvcc --version 获取 CUDA 版本
        result = subprocess.run(['nvcc', '--version'], capture_output=True, text=True)
        if result.returncode == 0: # 如果命令成功执行
            for line in result.stdout.splitlines():
                if 'release' in line: # 查找包含 'release' 的行
                    return line.split('release ')[1].split(',')[0] # 解析行，提取版本号
        return "CUDA not found or version not parsed" # 如果找不到 CUDA 或版本无法解析，设置提示信息
    except FileNotFoundError:
        return "CUDA not found" # 如果找不到 nvcc 命令，设置提示信息

# 获取 Python 版本的函数
def get_python_version():
    return platform.python_version() # 获取 Python 版本

# 获取 Conda 版本的函数
def get_conda_version():
    try:
        # 尝试使用 conda --version 获取 Conda 版本
        result = subprocess.run(['conda', '--version'], capture_output=True, text=True)
        if result.returncode == 0: # 如果命令成功执行
            return result.stdout.strip() # 返回 Conda 版本
        return "Conda not found or version not parsed" # 如果找不到 Conda 或版本无法解析，设置提示信息
    except FileNotFoundError:
        return "Conda not found" # 如果找不到 conda 命令，设置提示信息

# 获取物理磁盘空间信息的函数
def get_disk_space():
    try:
        total, used, free = shutil.disk_usage("/") # 获取根目录的磁盘使用情况
        total_gb = total / (1024**3) # 转换为 GB
        used_gb = used / (1024**3) # 转换为 GB
        free_gb = free / (1024**3) # 转换为 GB
        return f"Total: {total_gb:.2f} GB, Used: {used_gb:.2f} GB, Free: {free_gb:.2f} GB" # 格式化输出
    except Exception as e:
        return f"Could not retrieve disk info: {e}" # 如果获取信息出错，设置错误信息

# 获取环境信息
os_name = platform.system() # 获取操作系统名称
os_version = platform.release() # 获取操作系统版本
if os_name == "Linux":
    try:
        # 在 Linux 上尝试获取发行版和版本
        lsb_info = subprocess.run(['lsb_release', '-a'], capture_output=True, text=True)
        if lsb_info.returncode == 0: # 如果命令成功执行
            for line in lsb_info.stdout.splitlines():
                if 'Description:' in line: # 查找包含 'Description:' 的行
                    os_version = line.split('Description:')[1].strip() # 提取描述信息作为版本
                    break # 找到后退出循环
                elif 'Release:' in line: # 查找包含 'Release:' 的行
                     os_version = line.split('Release:')[1].strip() # 提取版本号
                     # 尝试获取 codename
                     try:
                         codename_info = subprocess.run(['lsb_release', '-c'], capture_output=True, text=True)
                         if codename_info.returncode == 0:
                             os_version += f" ({codename_info.stdout.split(':')[1].strip()})" # 将 codename 添加到版本信息中
                     except:
                         pass # 如果获取 codename 失败则忽略

    except FileNotFoundError:
        pass # lsb_release 可能未安装，忽略错误

full_os_info = f"{os_name} {os_version}" # 组合完整的操作系统信息
cpu_info = get_cpu_info() # 调用函数获取 CPU 信息和核心数量
memory_info = get_memory_info() # 调用函数获取内存信息
gpu_info = get_gpu_info() # 调用函数获取 GPU 信息和显存
cuda_version = get_cuda_version() # 调用函数获取 CUDA 版本
python_version = get_python_version() # 调用函数获取 Python 版本
conda_version = get_conda_version() # 调用函数获取 Conda 版本
disk_info = get_disk_space() # 调用函数获取物理磁盘空间信息


# 创建用于存储数据的字典
env_data = {
    "项目": [ # 项目名称列表
        "操作系统",
        "CPU 信息",
        "内存信息",
        "GPU 信息",
        "CUDA 信息",
        "Python 版本",
        "Conda 版本",
        "物理磁盘空间" # 添加物理磁盘空间
    ],
    "信息": [ # 对应的信息列表
        full_os_info,
        cpu_info,
        memory_info,
        gpu_info,
        cuda_version,
        python_version,
        conda_version,
        disk_info # 添加物理磁盘空间信息
    ]
}

# 创建一个 pandas DataFrame
df = pd.DataFrame(env_data)

# 打印表格
print("### 环境信息") # 打印标题
print(df.to_markdown(index=False)) # 将 DataFrame 转换为 Markdown 格式并打印，不包含索引

### 环境信息
| 项目         | 信息                                                               |
|:-------------|:-------------------------------------------------------------------|
| 操作系统     | Linux Ubuntu 22.04.4 LTS                                           |
| CPU 信息     | Intel(R) Xeon(R) CPU @ 2.20GHz (1 physical cores, 2 logical cores) |
| 内存信息     | 12.67 GB (Available: 8.10 GB)                                      |
| GPU 信息     | Tesla T4 (15360 MiB)                                               |
| CUDA 信息    | 12.5                                                               |
| Python 版本  | 3.12.12                                                            |
| Conda 版本   | Conda not found                                                    |
| 物理磁盘空间 | Total: 112.64 GB, Used: 44.60 GB, Free: 68.02 GB                   |


## 📦 第二步：安装依赖包

现在我们需要安装运行模型所需的关键 Python 包：

####  **vllm (0.12.0)**
- **作用**: 高性能大语言模型推理引擎
- **用途**: 加载和运行 DeepSeek 模型
- **优势**: 内存高效，推理速度快

### ⚡ 安装过程
下面的命令会安装所有必需的依赖包，请耐心等待安装完成。

In [2]:
!pip install \
    vllm==0.12.0

## 🚀 第三步：启动 VLLM 模型服务

现在我们将使用 VLLM 在后台启动 DeepSeek R1 蒸馏版模型服务。

### 🎯 VLLM 服务启动说明

#### 🔍 模型选择
- **模型来源**: [Hugging Face DeepSeek AI](https://huggingface.co/deepseek-ai/DeepSeek-R1#3-model-downloads)
- **当前模型**: `deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B`
- **参数规模**: 15亿参数，适合 T4 GPU 运行
- **替换选项**: 可以替换为其他 DeepSeek R1 系列模型

#### ⚙️ VLLM 参数解释
- `serve`: VLLM 的服务模式命令
- `--trust-remote-code`: 允许执行远程代码（模型配置）
- `--dtype half`: 使用半精度浮点数，节省显存
- `--max-model-len 16384`: 最大序列长度为 16K tokens
- `--tensor-parallel-size 1`: 使用单卡推理

#### 🔄 后台运行
模型将在后台启动，不会阻塞当前进程，这样我们可以继续执行其他代码。


In [3]:
import os
import subprocess

# 📝 可选：配置 Hugging Face 镜像源
# 如果在中国大陆访问 Hugging Face 较慢，可以启用下面这行
# os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"

# 🎯 模型配置
# 指定要使用的模型名称
# 这里使用的是 DeepSeek R1 的蒸馏版本，参数量为 15亿
model = 'deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B'

# 🔧 启动 VLLM 服务器
# 使用 subprocess.Popen 在后台启动服务，这样不会阻塞当前进程
print(f"🚀 正在启动 VLLM 服务，模型: {model}")
print("⏳ 首次运行需要下载模型，请耐心等待...")

vllm_process = subprocess.Popen([
    'vllm',                      # VLLM 命令
    'serve',                     # 服务模式
    model,                       # 模型名称
    '--trust-remote-code',       # 信任远程代码
    '--dtype', 'half',           # 使用半精度浮点数
    '--max-model-len', '4096',  # 最大序列长度
    '--tensor-parallel-size', '1', # 单卡推理
    '--gpu-memory-utilization', '0.72' # 降低 GPU 内存使用率
], stdout=subprocess.PIPE, stderr=subprocess.PIPE, start_new_session=True)

print("✅ VLLM 服务启动命令已执行，正在后台加载模型...")
print("📡 服务将在 http://localhost:8000 上运行")

🚀 正在启动 VLLM 服务，模型: deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B
⏳ 首次运行需要下载模型，请耐心等待...
✅ VLLM 服务启动命令已执行，正在后台加载模型...
📡 服务将在 http://localhost:8000 上运行


## 🔍 第四步：监控 vLLM 服务状态

由于 vLLM 在后台运行，我们需要监控其启动状态。

### 🎯 监控的重要性

#### 🔄 为什么需要监控？
- **异步启动**: vLLM 在后台启动，需要时间加载模型
- **状态确认**: 确保服务正常运行后再进行后续操作
- **错误诊断**: 及时发现和处理启动过程中的问题
- **资源管理**: 监控进程状态，避免资源泄漏

#### ⏱️ 启动时间说明
- **首次运行**: 需要下载模型文件，可能需要 5-10 分钟
- **后续运行**: 模型已缓存，启动时间约 1-2 分钟
- **检查频率**: 每 5 秒检查一次服务状态

#### 🚦 状态检查机制
- **健康检查**: 通过 HTTP 请求检查服务是否可用
- **进程监控**: 监控 vLLM 进程的运行状态
- **日志输出**: 显示启动过程中的关键信息

In [4]:
# 🔍 VLLM 服务监控函数
#
# 这个单元格定义了两个重要的监控函数：
# 1. check_vllm_status: 检查 VLLM 服务是否可用
# 2. monitor_vllm_process: 持续监控 VLLM 进程状态

import requests
import time
from typing import Tuple
import sys

def check_vllm_status(url: str = "http://localhost:8000/health") -> bool:
    """
    🏥 检查 VLLM 服务器健康状态

    参数:
        url: 健康检查的 URL 地址

    返回:
        bool: True 表示服务正常，False 表示服务不可用

    工作原理:
        向 VLLM 的健康检查端点发送 GET 请求
        如果返回 200 状态码，说明服务正常运行
    """
    try:
        response = requests.get(url, timeout=5)
        return response.status_code == 200
    except requests.exceptions.ConnectionError:
        return False
    except requests.exceptions.Timeout:
        return False
    except Exception:
        return False

def monitor_vllm_process(vllm_process: subprocess.Popen, check_interval: int = 5) -> Tuple[bool, str, str]:
    """
    📊 监控 VLLM 进程的启动状态

    参数:
        vllm_process: VLLM 进程对象
        check_interval: 检查间隔时间（秒）

    返回:
        Tuple[bool, str, str]: (是否成功, 标准输出, 标准错误)

    工作流程:
        1. 循环检查进程是否还在运行
        2. 定期检查服务健康状态
        3. 输出进程的日志信息
        4. 返回最终状态
    """
    print("🔍 开始 VLLM 服务器监控...")
    print("⏳ 正在等待服务启动，请耐心等待...")

    while vllm_process.poll() is None:  # 当进程仍在运行时
        # 检查服务是否已经可用
        if check_vllm_status():
            print("✅ VLLM 服务器已启动并运行！")
            print("🎉 服务地址: http://localhost:8000")
            return True, "", ""

        print("⏳ 等待 VLLM 服务器启动...")
        time.sleep(check_interval)

        # 检查并输出进程日志
        if vllm_process.stdout and vllm_process.stdout.readable():
            try:
                stdout = vllm_process.stdout.read1(1024).decode('utf-8')
                if stdout.strip():
                    print("📝 标准输出:", stdout.strip())
            except Exception:
                pass

        if vllm_process.stderr and vllm_process.stderr.readable():
            try:
                stderr = vllm_process.stderr.read1(1024).decode('utf-8')
                if stderr.strip():
                    print("⚠️ 标准错误:", stderr.strip())
            except Exception:
                pass

    # 如果到达这里，进程已结束（可能是错误）
    print("❌ VLLM 进程已结束")
    try:
        stdout, stderr = vllm_process.communicate(timeout=5)
        return False, stdout.decode('utf-8'), stderr.decode('utf-8')
    except Exception:
        return False, "", "进程通信超时"

In [5]:
# 🚀 执行 VLLM 服务监控
#
# 这个单元格的作用：
# 1. 调用监控函数，等待 VLLM 服务启动
# 2. 处理启动成功和失败的情况
# 3. 支持用户中断操作

print("🎯 开始监控 VLLM 服务启动状态...")
print("💡 提示：首次运行可能需要 5-10 分钟下载模型")
print("⌨️  按 Ctrl+C 可以中断监控（但不会停止 VLLM 服务）")

try:
    # 调用监控函数，等待服务启动
    success, stdout, stderr = monitor_vllm_process(vllm_process)

    if not success:
        print("\n❌ VLLM 服务器启动失败！")
        print("\n📋 完整标准输出:")
        print(stdout)
        print("\n🚨 完整标准错误:")
        print(stderr)
        print("\n🔧 可能的解决方案:")
        print("1. 检查 GPU 内存是否足够")
        print("2. 确认模型名称是否正确")
        print("3. 重新运行安装依赖包的单元格")
        sys.exit(1)
    else:
        print("\n🎉 VLLM 服务启动成功！")
        print("📡 API 服务地址: http://localhost:8000")
        print("📚 API 文档地址: http://localhost:8000/docs")
        print("✅ 现在可以继续运行后续单元格")

except KeyboardInterrupt:
    print("\n⚠️ 用户中断监控")
    print("💡 注意：VLLM 服务仍在后台运行")
    print("🔄 如果需要停止 VLLM 服务，请重启 Colab 运行时")

    # 可选：强制停止 VLLM 进程
    # 取消下面的注释可以在中断时停止服务
    # print("🛑 正在停止 VLLM 服务...")
    # vllm_process.terminate()
    # try:
    #     vllm_process.wait(timeout=5)
    #     print("✅ VLLM 服务已停止")
    # except subprocess.TimeoutExpired:
    #     vllm_process.kill()
    #     print("⚡ 强制终止 VLLM 服务")

    # 输出最终日志信息
    try:
        stdout, stderr = vllm_process.communicate(timeout=2)
        if stdout:
            print("\n📝 最终标准输出:")
            print(stdout.decode('utf-8'))
        if stderr:
            print("\n⚠️ 最终标准错误:")
            print(stderr.decode('utf-8'))
    except:
        print("📝 无法获取最终日志")

    sys.exit(0)

🎯 开始监控 VLLM 服务启动状态...
💡 提示：首次运行可能需要 5-10 分钟下载模型
⌨️  按 Ctrl+C 可以中断监控（但不会停止 VLLM 服务）
🔍 开始 VLLM 服务器监控...
⏳ 正在等待服务启动，请耐心等待...
✅ VLLM 服务器已启动并运行！
🎉 服务地址: http://localhost:8000

🎉 VLLM 服务启动成功！
📡 API 服务地址: http://localhost:8000
📚 API 文档地址: http://localhost:8000/docs
✅ 现在可以继续运行后续单元格


## 🔍 第五步：大模型推理测试

In [6]:
# 🧪 模型推理测试函数
#
# 这个单元格定义了两个核心函数：
# 1. ask_model: 发送问题并获取完整回答
# 2. stream_llm_response: 实现流式响应功能

import requests
import json
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from fastapi.responses import StreamingResponse

# 📝 定义请求数据模型
class QuestionRequest(BaseModel):
    """
    API 请求的数据模型

    属性:
        question (str): 用户提出的问题

    说明:
        使用 Pydantic 模型确保数据类型安全
        后续 FastAPI 会自动验证请求数据
    """
    question: str

def ask_model(question: str):
    """
    🤖 向 VLLM 模型发送问题并获取完整回答

    参数:
        question (str): 用户提出的问题

    返回:
        dict: 包含模型回答的 JSON 响应

    工作流程:
        1. 构造符合 OpenAI API 格式的请求
        2. 发送 POST 请求到 VLLM 服务
        3. 处理响应并返回结果
    """
    # VLLM 的 OpenAI 兼容 API 端点
    url = "http://localhost:8000/v1/chat/completions"

    # 设置请求头
    headers = {"Content-Type": "application/json"}

    # 构造请求数据（OpenAI 格式）
    data = {
        "model": model,  # 使用全局模型变量
        "messages": [
            {
                "role": "user",
                "content": question
            }
        ],
        "max_tokens": 2048,  # 最大生成长度
        "temperature": 0.7,  # 生成的随机性
        "top_p": 0.9         # 核采样参数
    }

    try:
        # 发送请求
        response = requests.post(url, headers=headers, json=data, timeout=60)
        response.raise_for_status()  # 检查 HTTP 错误
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"❌ 请求失败: {e}")
        return None

def stream_llm_response(question: str):
    """
    🌊 流式响应生成器 - 实时获取模型输出

    参数:
        question (str): 用户提出的问题

    生成:
        str: 逐行返回模型的生成内容

    特点:
        - 实时显示生成过程
        - 降低等待时间
        - 提供更好的用户体验
    """
    url = "http://localhost:8000/v1/chat/completions"
    headers = {"Content-Type": "application/json"}

    # 启用流式传输
    data = {
        "model": model,
        "messages": [{"role": "user", "content": question}],
        "stream": True,      # 🔥 关键：启用流式传输
        "max_tokens": 2048,
        "temperature": 0.7
    }

    try:
        with requests.post(url, headers=headers, json=data, stream=True, timeout=60) as response:
            response.raise_for_status()

            for line in response.iter_lines():
                if line:
                    # OpenAI 风格的流式响应以 "data: " 为前缀
                    decoded_line = line.decode("utf-8")
                    if decoded_line.startswith("data: "):
                        decoded_line = decoded_line[6:]  # 移除 "data: " 前缀
                    yield decoded_line + "\n"
    except requests.exceptions.RequestException as e:
        yield f"❌ 流式请求失败: {e}\n"



### 📝 两种推理方式对比

#### 1️⃣ **ask_model()** - 完整响应模式
- **特点**: 等待模型生成完整回答后一次性返回
- **返回类型**: `dict` (字典对象)
- **适用场景**:
  - 批量处理
  - 不需要实时反馈
  - 需要完整数据结构进行后续处理

#### 2️⃣ **stream_llm_response()** - 流式响应模式  
- **特点**: 实时逐块返回模型生成的内容
- **返回类型**: `generator` (生成器对象)
- **适用场景**:
  - 实时对话界面
  - 长文本生成
  - 需要即时反馈的交互场景

#### ⚠️ 常见错误
```python
# ❌ 错误用法：试图序列化生成器
result = stream_llm_response("问题")
print(json.dumps(result))  # 报错：Object of type generator is not JSON serializable

# ✅ 正确用法：遍历生成器
for chunk in stream_llm_response("问题"):
    # 处理每个数据块
    process(chunk)
```


In [7]:
# 🧪 测试基础推理功能
print("🧪 测试模型推理功能...")
print("📝 发送测试问题: 我是一名Java后端工程师，有5年工作经历，帮我制定一份Agentic AI学习计划")

try:
    result = ask_model("我是一名Java后端工程师，有5年工作经历，帮我制定一份Agentic AI学习计划")
    if result:
        print("\n✅ 模型推理成功！")
        print("📋 完整响应:")
        print(json.dumps(result, indent=2, ensure_ascii=False))

        # 提取并显示模型回答
        if "choices" in result and len(result["choices"]) > 0:
            answer = result["choices"][0]["message"]["content"]
            print(f"\n🤖 模型回答: {answer}")
    else:
        print("❌ 模型推理失败")
except Exception as e:
    print(f"❌ 测试过程中出现错误: {e}")

print("\n" + "="*50)
print("✅ 推理函数定义完成，可以继续下一步！")

🧪 测试模型推理功能...
📝 发送测试问题: 我是一名Java后端工程师，有5年工作经历，帮我制定一份Agentic AI学习计划

✅ 模型推理成功！
📋 完整响应:
{
  "id": "chatcmpl-82fcde2c8a813199",
  "object": "chat.completion",
  "created": 1769157859,
  "model": "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "好的，我现在需要帮助这位Java后端工程师制定一份Agentic AI学习计划。首先，我得了解Agentic AI是什么，以及他适用的领域。根据之前的信息，它是一种基于AI的工具，用于数据分析和数据驱动的决策。这可能适用于企业，比如制造业、医疗、金融等，因为这些领域需要快速分析数据来做出决策。\n\n这位工程师有5年的Java后端经验，说明他可能已经有了一定的Java编程基础，但可能在AI和数据科学方面还有提升的空间。因此，他的学习计划应该从基础开始，逐步提升，结合实际项目应用。\n\n首先，我应该考虑他的学习路径。可能需要先学习基础的Python和Java，然后是机器学习和深度学习，接着是数据工程和AI框架的使用，比如TensorFlow和PyTorch。此外，他可能需要学习一些高级的AI工具和框架，如Scikit-learn、XGBoost、Keras、TF-Flux等。此外，持续学习和实践也是很重要的，可能需要参与项目和社区贡献。\n\n接下来，我应该考虑他的时间分配。假设他每天有8小时的工作时间，那么5年的工作时间大约是360天。这意味着他需要合理分配时间，可能每周学习几小时，集中精力在特定领域上。可能需要制定一个分阶段的学习计划，每个阶段分配一定的学习时间，然后进行实践和项目。\n\n然后，我需要考虑他的技能需求。他需要掌握Python和Java的编程基础，了解数据科学的基本概念，比如数据清洗、特征工程、数据可视化等。此外，了解机器学习的基本概念，比如监督学习、无监督学习、强化学习

In [8]:
# 💡 实用工具：流式响应处理器
#
# 这个函数封装了流式响应的处理逻辑，可以方便地复用

def process_stream_response(question: str, show_chunks=False):
    """
    处理流式响应的工具函数

    参数:
        question: 要询问的问题
        show_chunks: 是否显示每个数据块的详细信息

    返回:
        str: 完整的模型回答
    """
    print(f"🔍 问题: {question}\n")
    print("💬 回答: ", end="", flush=True)

    full_response = ""

    try:
        for chunk in stream_llm_response(question):
            if not chunk.strip():
                continue

            if chunk.strip() == "[DONE]":
                break

            try:
                chunk_data = json.loads(chunk.strip())

                if "choices" in chunk_data and len(chunk_data["choices"]) > 0:
                    delta = chunk_data["choices"][0].get("delta", {})
                    content = delta.get("content", "")

                    if content:
                        print(content, end="", flush=True)
                        full_response += content

                    # 可选：显示数据块详情
                    if show_chunks and content:
                        print(f"\n[Chunk: {len(content)} chars]", end="")

            except json.JSONDecodeError:
                if "❌" in chunk:
                    print(f"\n{chunk}")

        print("\n")  # 换行
        return full_response

    except Exception as e:
        print(f"\n\n❌ 错误: {e}")
        return None

# 🧪 使用示例
print("="*60)
print("🎯 示例：使用流式响应处理器")
print("="*60 + "\n")

answer = process_stream_response("请用一句话介绍什么是Agentic AI")

if answer:
    print(f"✅ 成功获取完整回答（共 {len(answer)} 字符）")


🎯 示例：使用流式响应处理器

🔍 问题: 请用一句话介绍什么是Agentic AI

💬 回答: 嗯，用户想了解什么是Agentic AI。首先，我得弄清楚Agentic是什么。从名字来看，“Agentic”可能与“agent”有关，所以应该是与智能体相关的技术。AI通常指人工智能，所以Agentic AI应该是指结合了智能体和人工智能的技术。

接下来，我需要解释Agentic AI是什么。它应该指的是能够自主行动并与环境互动的智能系统。可能和强化学习、多 Agent 系统等技术有关。用户可能对AI有一定了解，但需要更详细的解释。

用户可能想知道Agentic AI在哪些领域应用，比如机器人控制、游戏开发、物流优化等。所以，我应该提到这些应用，说明Agentic AI如何在这些领域中发挥作用，比如通过控制智能体来实现目标。

另外，用户可能关心Agentic AI的优势，比如自主性和适应性。这样用户能更好地理解它的价值和适用场景。我应该强调Agentic AI如何提升AI系统的行为和决策能力。

最后，总结一下，Agentic AI是人工智能领域中的一个重要概念，结合了智能体和环境，通过自主行动和学习来提升系统性能。用户应该能够清楚地理解并应用这个概念。
</think>

Agentic AI 是一种结合了智能体和人工智能的技术，指的是一种能够自主行动并与环境互动的智能系统。它通过强化学习、多 Agent 系统和感知反馈等方法，提升 AI 系统的行为和决策能力。Agentic AI 在机器人控制、游戏开发、物流优化等领域有广泛应用，通过智能体的自主学习和主动适应，显著提升了系统的行为和任务完成能力。

✅ 成功获取完整回答（共 665 字符）


In [11]:
# 🧪 测试流式推理功能
print("🧪 测试模型流式推理功能...")
print("📝 发送测试问题: 我是一名Java后端工程师，有5年工作经历，帮我制定一份Agentic AI学习计划")
print("\n🌊 开始接收流式响应...\n")

try:
    # stream_llm_response 返回的是生成器，需要逐行处理
    full_response = ""
    chunk_count = 0

    for chunk in stream_llm_response("我是一名Java后端工程师，有5年工作经历，帮我制定一份Agentic AI学习计划"):
        chunk_count += 1

        # 跳过空行
        if not chunk.strip():
            continue

        # 检查是否是结束标记
        if chunk.strip() == "[DONE]":
            print("\n\n✅ 流式响应完成！")
            break

        try:
            # 解析 JSON 数据
            chunk_data = json.loads(chunk.strip())

            # 提取内容增量
            if "choices" in chunk_data and len(chunk_data["choices"]) > 0:
                delta = chunk_data["choices"][0].get("delta", {})
                content = delta.get("content", "")

                if content:
                    # 实时打印内容（不换行）
                    print(content, end="", flush=True)
                    full_response += content
        except json.JSONDecodeError:
            # 如果不是有效的 JSON，可能是错误消息
            if "❌" in chunk:
                print(chunk)

    print(f"\n\n📊 统计信息:")
    print(f"  - 接收到的数据块数: {chunk_count}")
    print(f"  - 完整回答长度: {len(full_response)} 字符")
    print("\n✅ 流式推理测试成功！")

except Exception as e:
    print(f"\n❌ 测试过程中出现错误: {e}")
    import traceback
    traceback.print_exc()

print("\n" + "="*50)
print("✅ 流式推理函数测试完成！")

🧪 测试模型流式推理功能...
📝 发送测试问题: 我是一名Java后端工程师，有5年工作经历，帮我制定一份Agentic AI学习计划

🌊 开始接收流式响应...

好，我现在需要帮用户制定一份Agentic AI学习计划。用户是Java后端工程师，有5年经验。首先，我得考虑他的工作背景，可能在技术方面比较强，但可能对AI不太熟悉，所以计划需要逐步深入。

首先，用户需要学习AI基础知识，所以我要先列出基础课程。Python和TensorFlow应该是核心，然后是深度学习的基础，比如神经网络和机器学习理论。接着，可能需要了解NLP和计算机视觉，这些都是AI中的热门领域，特别是现在AI在这些领域的应用很流行。

然后，用户可能还需要了解数据科学和机器学习的实践，比如机器学习的基本概念和应用案例，这能帮助他将理论应用到实际项目中。同时，NLP和计算机视觉的实用课程也很重要，因为这些领域现在很受欢迎。

接下来是项目实践，这可能包括PyTorch框架，因为它是更现代的替代方案，和TensorFlow不同。然后是强化学习，这部分比较难，但很重要。用户可能需要从基础到高级，所以需要分阶段学习，逐步深入。

最后，可能需要学习高级技术，比如微服务和AI模型部署，这对用户来说是未来的重要方向。用户可能还想了解最新的趋势，所以还需要涵盖当前AI和机器学习的发展动态。

整个计划需要分阶段，分阶段学习，每个阶段里有具体的课程和项目。用户可能需要每天学习，同时有时间处理项目，这样能更有效率。另外，用户可能还需要考虑时间管理，比如每天的学习时间，以及如何分配项目时间，避免压力。

在制定计划的时候，我需要确保内容全面，涵盖基础知识、应用、项目实践和未来趋势，这样用户的学习会更全面。同时，要提醒用户要关注最新动态，保持学习的持续性。

总的来说，我需要分步骤列出课程和项目，确保每个阶段都有对应的课程和项目，帮助用户逐步深入学习AI技术。这样用户才能在短时间内掌握足够的知识和技能，为未来的职业发展打下坚实的基础。
</think>

好的！以下是一份针对你背景的AI学习计划，涵盖基础知识、应用、项目实践以及未来趋势。计划分为多个阶段，每个阶段包含相关课程和项目：

---

### **第一阶段：AI基础知识（Python、TensorFlow、深度学习基础）**
#### **课程**

### 📋 API 响应示例

### 🤖 完整响应格式

当你调用 `/api/v1/generate-response` 端点时，会收到如下格式的 JSON 响应：

```
{
  "response": {
    "id": "chatcmpl-13e29c35212b486ead18d91aa0668886",
    "object": "chat.completion",
    "created": 1752386782,
    "model": "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B",
    "choices": [
      {
        "index": 0,
        "message": {
          "role": "assistant",
          "reasoning_content": null,
          "content": "好，用户发来“呦呦鹿鸣”这个词，看起来像是在玩手机的语音合成功能。我应该回复用户一个友好又有趣的回应，比如“嗯，看到你这么说，我好像也听到了，鹿鸣的声音很温柔啊！”这样既回应了他们的提问，又让语气更生动。另外，可以用一些轻松的语气，让用户觉得有趣又不觉得压力。可能用户想了解这句话的含义，或者只是想在聊天。所以，我需要保持自然，不显得太生硬。另外，可能用户是想测试一下语音合成的功能，或者是想了解一些有趣的话题。不管怎样，回应要友好，同时带点趣味，让用户感到愉快。\n</think>\n\n嗯，看到你这么说，我好像也听到了，鹿鸣的声音很温柔啊！",
          "tool_calls": []
        },
        "logprobs": null,
        "finish_reason": "stop",
        "stop_reason": null
      }
    ],
    "usage": {
      "prompt_tokens": 9,
      "total_tokens": 176,
      "completion_tokens": 167,
      "prompt_tokens_details": null
    },
    "prompt_logprobs": null,
    "kv_transfer_params": null
  }
}
```

#### 📊 响应结构说明
- **id**: 请求的唯一标识符
- **object**: 响应对象类型
- **created**: 响应创建时间戳
- **model**: 使用的模型名称
- **choices**: 模型生成的选择列表
  - **index**: 选择的索引
  - **message**: 消息内容
    - **role**: 角色（assistant）
    - **content**: 模型生成的回答
  - **finish_reason**: 完成原因（stop 表示正常结束）
- **usage**: 令牌使用统计
  - **prompt_tokens**: 输入令牌数
  - **completion_tokens**: 生成令牌数
  - **total_tokens**: 总令牌数

### 🌊 流式响应格式

当你调用 `/api/v1/generate-response-stream` 端点时，会收到一系列 JSON 对象，每个对象代表生成过程中的一个步骤：



```
{"id":"chatcmpl-72594106be2541269cc68e8b37123051","object":"chat.completion.chunk","created":1752386825,"model":"deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B","choices":[{"index":0,"delta":{"role":"assistant","content":""},"logprobs":null,"finish_reason":null}]}
{"id":"chatcmpl-72594106be2541269cc68e8b37123051","object":"chat.completion.chunk","created":1752386825,"model":"deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B","choices":[{"index":0,"delta":{"content":"嗯"},"logprobs":null,"finish_reason":null}]}
{"id":"chatcmpl-72594106be2541269cc68e8b37123051","object":"chat.completion.chunk","created":1752386825,"model":"deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B","choices":[{"index":0,"delta":{"content":"，"},"logprobs":null,"finish_reason":null}]}
{"id":"chatcmpl-72594106be2541269cc68e8b37123051","object":"chat.completion.chunk","created":1752386825,"model":"deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B","choices":[{"index":0,"delta":{"content":"我现在"},"logprobs":null,"finish_reason":null}]}
{"id":"chatcmpl-72594106be2541269cc68e8b37123051","object":"chat.completion.chunk","created":1752386825,"model":"deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B","choices":[{"index":0,"delta":{"content":"在"},"logprobs":null,"finish_reason":null}]}
{"id":"chatcmpl-72594106be2541269cc68e8b37123051","object":"chat.completion.chunk","created":1752386825,"model":"deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B","choices":[{"index":0,"delta":{"content":"学习"},"logprobs":null,"finish_reason":null}]}
{"id":"chatcmpl-72594106be2541269cc68e8b37123051","object":"chat.completion.chunk","created":1752386825,"model":"deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B","choices":[{"index":0,"delta":{"content":"大"},"logprobs":null,"finish_reason":null}]}
{"id":"chatcmpl-72594106be2541269cc68e8b37123051","object":"chat.completion.chunk","created":1752386825,"model":"deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B","choices":[{"index":0,"delta":{"content":"模型"},"logprobs":null,"finish_reason":null}]}
{"id":"chatcmpl-72594106be2541269cc68e8b37123051","object":"chat.completion.chunk","created":1752386825,"model":"deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B","choices":[{"index":0,"delta":{"content":"，"},"logprobs":null,"finish_reason":null}]}
{"id":"chatcmpl-72594106be2541269cc68e8b37123051","object":"chat.completion.chunk","created":1752386825,"model":"deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B","choices":[{"index":0,"delta":{"content":"比如"},"logprobs":null,"finish_reason":null}]}
...
[DONE]
```



#### 📡 流式响应特点
- **实时性**: 逐步返回生成内容，无需等待完整回答
- **低延迟**: 用户可以立即看到模型开始生成
- **更好体验**: 适合长文本生成和实时对话

#### 🔄 流式数据格式
每行数据都是一个独立的 JSON 对象，包含：
- **id**: 请求标识符（整个流中保持一致）
- **object**: "chat.completion.chunk"
- **created**: 时间戳
- **model**: 模型名称
- **choices**: 当前生成的内容块
  - **index**: 选择索引
  - **delta**: 增量内容
    - **content**: 新生成的文本片段
  - **finish_reason**: 结束原因（null 表示继续，"stop" 表示结束）

#### 💡 使用建议
- 适合需要实时反馈的应用场景
- 可以实现打字机效果的用户界面
- 对于长文本生成特别有用

## 🔍 第六步： 终止 vLLM

In [10]:
# 如果要终止vLLM，可以执行如下代码
vllm_process.terminate()
vllm_process.wait()  # Wait for process to terminate

1